In [3]:
import pandas as pd
import os

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

data = pd.read_excel(url)
data

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.isnull().sum()

Date                25
Year                27
Type                43
Country             75
State              507
Location           590
Activity           611
Name               245
Sex                604
Age               3019
Injury              60
Unnamed: 11        587
Time              3551
Species           3157
Source              44
pdf                170
href formula       150
href               173
Case Number        171
Case Number.1      172
original order     170
Unnamed: 21       6968
Unnamed: 22       6967
dtype: int64

In [12]:
#we noticed that the Species collumn has an empty space after the last letter, so we'll remove the space and save that to news dataset
data = data.rename(columns = {"Species ": "Species"})
data

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:


# we will chose certain variables that we think it's more important for identifying our business. and we chose these 7 collumns: 



selected_columns = data[["Date", "Year", "Country", "Location", "Injury", "Species", "Activity"]]
selected_columns

,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024.0,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024.0,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024.0,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024.0,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024.0,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing
...,...,...,...,...,...,...,...
6964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6965,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6966,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6967,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# clean the selected columns from NaN

selected_columns.dropna(how='all')

,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024.0,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024.0,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024.0,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024.0,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024.0,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing
...,...,...,...,...,...,...,...
6939,Before 1903,0.0,AUSTRALIA,Roebuck Bay,FATAL,NaN,Diving
6940,Before 1903,0.0,AUSTRALIA,NaN,FATAL,NaN,Pearl diving
6941,1900-1905,0.0,USA,Ocracoke Inlet,FATAL,NaN,Swimming
6942,1883-1889,0.0,PANAMA,"Panama Bay 8ºN, 79ºW",FATAL,NaN,NaN


In [40]:
# delete the NA from any raw and column 
selected_columns = selected_columns.dropna(subset = ['Year'])
selected_columns
df= selected_columns
df


,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024.0,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024.0,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024.0,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024.0,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024.0,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing
...,...,...,...,...,...,...,...
6939,Before 1903,0.0,AUSTRALIA,Roebuck Bay,FATAL,NaN,Diving
6940,Before 1903,0.0,AUSTRALIA,NaN,FATAL,NaN,Pearl diving
6941,1900-1905,0.0,USA,Ocracoke Inlet,FATAL,NaN,Swimming
6942,1883-1889,0.0,PANAMA,"Panama Bay 8ºN, 79ºW",FATAL,NaN,NaN


In [43]:
# correct the way the Years are written from for example 2023.0 to 2023 (from strin to integer )



df['Year'] = df['Year'].astype(int)
df



,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing
...,...,...,...,...,...,...,...
6939,Before 1903,0,AUSTRALIA,Roebuck Bay,FATAL,NaN,Diving
6940,Before 1903,0,AUSTRALIA,NaN,FATAL,NaN,Pearl diving
6941,1900-1905,0,USA,Ocracoke Inlet,FATAL,NaN,Swimming
6942,1883-1889,0,PANAMA,"Panama Bay 8ºN, 79ºW",FATAL,NaN,NaN


In [60]:
df= df[(df['Year'] >= 2015)]
df

,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing
...,...,...,...,...,...,...,...
421,12-Jan-2020,2020,THAILAND,Nang Thong Beach,Calf bitten,NaN,Swimming
422,07-Jan-2020,2020,AUSTRALIA,North West Island,Lacerations to leg,Lemon shark,Swimming
423,05-Jan-2020,2020,AUSTRALIA,Cull Island / Esperance,FATAL,White shark,Scuba diving
424,03-Jan-2020,2020,NEW CALEDONIA,Calana,Right leg bitten,NaN,NaN


In [61]:
df['Year'].value_counts()

Year
2021    109
2023    108
2020    101
2022     98
2024     10
Name: count, dtype: int64

In [62]:
df.Country.value_counts()

Country
USA                           202
AUSTRALIA                     101
BAHAMAS                        15
SOUTH AFRICA                   12
NEW ZEALAND                    10
NEW CALEDONIA                  10
BRAZIL                          8
EGYPT                           8
FRENCH POLYNESIA                8
MEXICO                          7
THAILAND                        4
BELIZE                          3
ECUADOR                         2
JAMAICA                         2
ENGLAND                         2
FIJI                            2
MALDIVES                        2
CUBA                            2
SPAIN                           2
COLOMBIA                        1
INDONESIA                       1
ST KITTS / NEVIS                1
JORDAN                          1
CANADA                          1
British Overseas Territory      1
New Zealand                     1
COSTA RICA                      1
COLUMBIA                        1
INDIA                           1
TRINID

In [63]:
df.loc[data['Country'] == 'USA']

,Date,Year,Country,Location,Injury,Species,Activity
1,04 Mar 2024,2024,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
10,30 Dec-2023,2023,USA,"Baby Beach, Maui",FATAL,NaN,Surfing
24,05 Nov-2023,2023,USA,"Juno Beach, Palm Beach County",Lacerations to right forearm,NaN,Swimming
29,25 Oct 2023,2023,USA,"Pua’ena Point, Haleiwa, Oahu",Bite to right thigh,8' tiger shark,Surfing
...,...,...,...,...,...,...,...
408,27-Mar-2020,2020,USA,"Pleasure Point, Santa Cruz County","No injury, paddleboard bitten",NaN,Stand-Up Paddleboarding
409,22-Mar-2020,2020,USA,"New Smyrna Beach, Volusia County",Left foot bitten,NaN,Surfing
410,22-Mar-2020,2020,USA,"New Smyrna Beach, Volusia County",Left foot bitten when he jumped on the shark ...,NaN,Surfing
415,05-Feb-2020,2020,USA,Wailea,"No injury, but paddleboard bitten",Tiger shark,Stand-Up Paddle boarding


In [59]:
df['Location'].value_counts()

Location
New Smyrna Beach, Volusia County     17
Cocoa Beach, Brevard  County          6
Ponce Inlet, Volusia County           6
Myrtle Beach, Horry County            5
Anna Maria Island, Manatee County     4
                                     ..
Seacliff                              1
Kauai                                 1
Horn Island                           1
Cat Island                            1
Esperance                             1
Name: count, Length: 353, dtype: int64

In [64]:
df['Injury'].value_counts()

Injury
FATAL                                                                43
Foot bitten                                                           9
Leg bitten                                                            8
Lacerations to leg                                                    4
Right leg bitten                                                      4
                                                                     ..
Lower left leg severely bitten, necessitating surgical amputation     1
Injury to upper lip                                                   1
Minor bite to left  foot                                              1
Four-inch cut to right foot                                           1
Probable drowning and scavenging                                      1
Name: count, Length: 310, dtype: int64